In [199]:
import pandas as pd
import geopandas as gpd

In [200]:
# only keep cycleways

In [201]:
run = 2

In [202]:
gdf = gpd.read_file(f'/Users/alexandra/Documents/GitHub/road_network_classification/data/berlin_prio_vset/run{run}/berlin_ways_pred.shp', dtype={'ID': int}, crs=25833)

In [203]:
gdf.columns

Index(['OGC_FID', 'ID', 'ELEM_NR', 'IST_RADVOR', 'ZUSTAND', 'ZUSTAND_T',
       'VERSION', 'SOLL_RVA', 'SOLL_FASTR', 'SOLL_ASPH', 'POTENZIAL',
       'QUICKWIN', 'ANMERKUNG', 'MAß_NR', 'MAß_SEITE', 'STEP_STUFE',
       'UKD__IN_T', 'AKTUELLES', 'BEZIRK', 'LäNGE', 'CHECKED', 'TYPE_GROUN',
       'QUALITY_GR', 'AVG_QUALIT', 'WAY_LENGTH', 'N_SEGMENTS', 'ROAD_TYPE',
       'TYPE_PRED', 'AVG_CLASS_', 'VOTE_COUNT', 'MIN_CAPTUR', 'MAX_CAPTUR',
       'geometry'],
      dtype='object')

In [204]:
gdf['ROAD_TYPE_refined'] = gdf['ROAD_TYPE'].apply(lambda x: 'cycleway' if x.startswith('cycleway') else x)

In [205]:
gdf.sort_values(['ID', 'N_SEGMENTS'], ascending=False, inplace=True)
gdf = gdf.groupby(['ID', 'ROAD_TYPE_refined']).first().reset_index()

In [206]:
# Define the custom order for the ROAD_TYPE column
road_type_order = ['cycleway', 'road', 'pedestrian', 'other']
# Convert the ROAD_TYPE column to a categorical type with the specified order
gdf['ROAD_TYPE_refined'] = pd.Categorical(gdf['ROAD_TYPE_refined'], categories=road_type_order, ordered=True)
# Sort the DataFrame by ID and the custom ordered ROAD_TYPE
gdf.sort_values(['ID', 'ROAD_TYPE_refined'], inplace=True)
gdf['ROAD_TYPE_refined'] = gdf['ROAD_TYPE_refined'].astype(str)

In [207]:
gdf = gdf.groupby('ID').first().reset_index()

In [208]:
gdf.crs = 25833

In [209]:
gdf['ID'] = gdf['ID'].astype(int)

In [210]:
gdf.to_file(f'/Users/alexandra/Documents/GitHub/road_network_classification/data/berlin_prio_vset/run{run}/berlin_ways_pred_cycleways.shp')

/var/folders/m7/jb7xk8h91p541g43zv2r8l740000gn/T/ipykernel_5373/2255434053.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(f'/Users/alexandra/Documents/GitHub/road_network_classification/data/berlin_prio_vset/run{run}/berlin_ways_pred_cycleways.shp')
